In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
import unicodedata
from ast import literal_eval
import numpy as np

Importamos csv

In [5]:
df_1 = pd.read_csv("DATA_EDA.csv",  index_col=0)

In [6]:
df1 = df_1[["title", "genres", "overview","popularity" ]]


Configuración de funciones para datos de entrenamiento de ML Conseguir una sopa de letras

In [ ]:
df1.title = df1.title.str.lower()# convierte todos los títulos a minúsculas
df1.genres = df1.genres.str.replace(" ","")# elimina los espacios en blanco
def safe_literal_eval(value):#función verifica si el valor es una cadena no nula 
    if isinstance(value, str) and not pd.isna(value):#garantiza que no se produzca un error al evaluar los valores nan 
        return literal_eval(value)
    return []
df1.genres = df1.genres.apply(safe_literal_eval)  # Establecer el tipo de datos correcto en la columna genres
df1["features"] = df1.genres  # crea una nueva columna


Conseguir una sopa de letras

In [ ]:
df1['genres'] = df1['genres'].apply(lambda x: ' '.join(x))  # Unir elementos de "genres"
df1['features'] = df1['genres'] + ' ' + df1['overview']  # Unir elementos de "genres" y "overview" en la columna "features"(caracteristicas)

In [9]:
df1 = df1.sort_values("popularity", ascending=False)  # Ordenar por popularidad en orden descendente
df1.drop(["genres", "overview", "popularity"], inplace=True, axis=1)  # Eliminar columnas que no se utilizarán en el modelo de ML
df1["features"] = df1["features"].str.lower()  # Normalizar los valores en la columna "features" a minúsculas
df1.dropna(subset=["features"], inplace=True) #Elimino filas vacias
df1.reset_index(drop=True, inplace=True)  # Restablecer el índice del DataFrame

In [10]:
df1.head(10)

,title,features
0,minions,family animation adventure comedy minions stua...
1,wonder woman,action adventure fantasy an amazon princess co...
2,beauty and the beast,family fantasy romance a live-action adaptatio...
3,baby driver,action crime after being coerced into working ...
4,big hero 6,adventure family animation action comedy the s...
5,deadpool,action adventure comedy deadpool tells the ori...
6,guardians of the galaxy vol. 2,action adventure comedy sciencefiction the gua...
7,avatar,action adventure fantasy sciencefiction in the...
8,john wick,action thriller ex-lunatic john wick comes off...
9,gone girl,mystery thriller drama with his wife's disappe...


Exportar conjunto de datos para datos de entrenamiento de ML

In [11]:
df1.to_csv("API1_fucnion7.csv")#exportar dataset

In [12]:
df = pd.read_csv("API1_fucnion7.csv", index_col=0)

In [ ]:
df

,title,features
0,minions,f a m i l y a n i m a t i o n a d v e n t ...
1,wonder woman,a c t i o n a d v e n t u r e f a n t a s ...
2,beauty and the beast,f a m i l y f a n t a s y r o m a n c e a ...
3,baby driver,a c t i o n c r i m e after being coerced in...
4,big hero 6,a d v e n t u r e f a m i l y a n i m a t ...
...,...,...
44399,hellyys,c o m e d y finnish soft erotic movie
44400,altar of fire,d o c u m e n t a r y this film records a 12 d...
44401,vehshi jatt,"classic indian film starring aasia, afzaal ah..."
44402,raakh,after witnessing the public humiliation &amp;...


In [13]:
ml = df.head(10000)  # Utilizar una muestra debido al costo computacional excesivo si se utiliza todo el conjunto de datos

tfidf = TfidfVectorizer(stop_words="english", max_features=10000)  # Configuración del vector tf-idf, elimina las palabras comunes en inglés y limita el número de filas a tomar
tfidf_matrix = tfidf.fit_transform(ml["features"])  # Configuración del vectorizador tf-idf con datos
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)  # Modelo de entrenamiento con los datos proporcionados


In [18]:

#@app.get('/recomendacion/{titulo}')    
def recomendacion(titulo: str):
    '''Ingresas un nombre de película y te recomienda las similares en una lista'''

    # Normalizar y formatear el título de la película
    titulo = titulo.lower().strip()
    titulo = unicodedata.normalize('NFKD', titulo).encode('ascii', 'ignore').decode('utf-8', 'ignore')

    # Buscar el índice de la película en el DataFrame ml
    idx = ml[ml["title"] == titulo].index

    # Verificar si se encontró la película en el conjunto de datos
    if len(idx) == 0:
        recommendations = ["Datos no disponibles"]
    else:
        idy = idx[0]

        # Obtener los puntajes de similitud de la película con todas las demás películas
        sim_scores = cosine_sim[idy]

        # Obtener los índices de las películas más similares
        top_indices = np.argsort(sim_scores)[::-1][1:6]

        # Obtener los títulos de las películas recomendadas
        recommendations = list(ml['title'].iloc[top_indices].str.title())

    return {'lista_recomendada': recommendations}


In [20]:
recomendacion("superman")

{'lista_recomendada': ['All Star Superman',
  'Superman Ii',
  'Superman Returns',
  'Superman Iv: The Quest For Peace',
  'The Freshman']}